In [1]:
import emapy as epy
import sys;
import pandas as pd

In [2]:
locationAreaBoundingBox = (41.3248770036,2.0520401001,41.4829908452,2.2813796997)
epy.divideBoundingBoxBySurfaceUnitSavedGeoJSON(
    locationAreaBoundingBox, 
    1, 
    1, 
    '../data/processed/surfaceBarcelona')
surface = epy.getDatabase(
    'surfaceBCN', 
    'geojson',
    '../data/processed/surfaceBarcelona.geojson', 
    '',
    True,
    0,
    1,
    'id')  

In [3]:
locationAreaBoundingBox = (41.3248770036,2.0520401001,41.4829908452,2.2813796997)

allData = epy.getDatabaseFromOSM(
    'restaurantes', 
    'amenity',
    False,
    True,
    locationAreaBoundingBox,
    'bar')

numJumps = 1

T = [[boxId, 
      data['properties'], 
      data['geometry'], 
      epy.getLessDistanceInKmBtwnCoordAndInfoStructureWithJumps(
        data["geometry"][0],
        data["geometry"][1],
        allData,
        numJumps, 
        True)[0]] 
     for boxId in surface[1] 
     for data in allData 
     if epy.coordInsidePolygon(data["geometry"][0], 
                               data["geometry"][1], 
                               epy.transformArrYXToXY(surface[1][boxId]))]

     


In [4]:
df = pd.DataFrame({'id' : [], 'data': []})

allId = dict()
allValue = dict()

for data in T:
    key = int(float(data[0]))
    if key in allId:
        allId[key] += 1
        allValue[key] += data[3]
    else:
        allId[key] = 1
        allValue[key] = data[3]

print allId     
        
for boxId in surface[1]:
    key = int(float(boxId))
    if key in allId:
        row = [key, allValue[key] * 1.0 / allId[key]]  
        df.loc[len(df), ['id', 'data']] = row  
    else:
        df.loc[len(df), ['id', 'data']] = [key,0]


In [5]:
map = epy.mapCreation(41.388790,2.158990)
epy.mapChoropleth(map, 
                 '../data/processed/surfaceBarcelona.geojson',
                 'feature.properties.id',
                 df,
                 'id',
                 'data',
                 'YlGn',
                 0.7,
                 0.3,
                 [],
                 'bars / barri')

In [6]:
dataNames = []
idNodes = []
color = 'blue'
for data in allData:
    if data['type'].strip().lower() == 'point': 
        prop = data['properties']
        name = ''
        if 'name' in prop:
            name = prop['name']
            dataNames.append(name)
        idNode = str(data['geometry'][0]) +  str(data['geometry'][1]) + name
        
        if idNode not in idNodes:
            idNodes.append(idNode)
            epy.mapAddMarker(
                map,
                data['geometry'][0],
                data['geometry'][1],
                'glyphicon-glass',
                color,
                name)


In [7]:
epy.mapSave(map, '../reports/maps/mapOfBarsDistanceUS.html')
map